## 참고 사이트
* 데이터 삽입: https://velog.io/@acdongpgm/DB.-%EB%B2%A1%ED%84%B0-%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B2%A0%EC%9D%B4%EC%8A%A4Vector-database-ChromaDB-%EC%82%AC%EC%9A%A9%ED%95%B4%EB%B3%B4%EA%B8%B0
* 모델 허깅페이스: https://huggingface.co/kakaobank/kf-deberta-base
* https://blog.naver.com/htk1019/223493142877

In [ ]:
pip install transformers

     |████████████████████████████████| 9.3 MB 2.1 MB/s eta 0:00:01
     |████████████████████████████████| 411 kB 64.7 MB/s eta 0:00:01
You should consider upgrading via the '/Users/cauhike/Documents/GitHub/rag/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install torch torchvision

     |████████████████████████████████| 61.0 MB 36.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 23.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 36.6 MB/s eta 0:00:01
You should consider upgrading via the '/Users/cauhike/Documents/GitHub/rag/env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from transformers import AutoModel, AutoTokenizer

model = AutoModel.from_pretrained("kakaobank/kf-deberta-base")
tokenizer = AutoTokenizer.from_pretrained("kakaobank/kf-deberta-base")

# text = "카카오뱅크와 에프엔가이드가 금융특화 언어모델을 공개합니다."
# tokens = tokenizer.tokenize(text)
# print(tokens)

# inputs = tokenizer(text, return_tensors="pt")
# model_output = model(**inputs)
# print(model_output)

In [8]:
from dotenv import load_dotenv
import os
import uuid
import chromadb
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_community.embeddings import OllamaEmbeddings
from langchain.embeddings.openai import OpenAIEmbeddings
from chromadb.utils import embedding_functions
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from kiwipiepy import Kiwi
import gradio as gr
from tqdm import tqdm
import time
import csv
import torch

## database 연결하고 collection 생성

In [4]:
## 컬렉션 생성/연결하기
import chromadb
client = chromadb.PersistentClient('chroma/')
client.list_collections()

[Collection(name=ten_files_openai_3072), Collection(name=ten_files_KF-DeBERTa)]

In [38]:
collection = client.get_or_create_collection(name="ten_files_KF-DeBERTa")
collection.peek()

{'ids': [],
 'embeddings': [],
 'metadatas': [],
 'documents': [],
 'uris': None,
 'data': None}

In [34]:
## 컬렉션 삭제하기
client.delete_collection(
    name="ten_files_KF-DeBERTa"
)

## 데이터 불러오기

In [5]:
## 한번에 여러 파일 불러오기
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader
loader = DirectoryLoader(path='../data', glob='*.pdf', loader_cls=PyMuPDFLoader, show_progress=True, use_multithreading=True)
data = loader.load()
data

100%|██████████| 10/10 [00:01<00:00,  6.75it/s]


[Document(page_content='Journal of Cadastre & Land InformatiX Vol.52 No.1 (2022)   57\n주소의 국제표준화 동향분석 및 시사점*\n- ISO 19160-2를 중심으로 -\nThe trend of international address standardization and implications\n- with a focus on ISO 19160-2 -\n김성훈** · 김보은*** · 원석환****\nKim, Sung-Hoon · Kim, Bo-Eun · Won, Seok-Hwan\nAbstract\n \nThis study analyzed the status of international address standardization and reviewed responses \nof nations contained in the ISO/CD 19160-2-N5652 standard document. It identified problems in \nKorea’s responses to the standard document, and presented ways of improvement. International \nstandard procedures for ISO 19160-2 are underway with the goal to its completion by July 2023. \nTo create an international standard, continuous monitoring and responses are required, and efforts \nto reflect the strength of the Korean addressing system on the new standard is needed. \nKeywords: International Standard, ISO 19160-2, Address, Road Name Address\n1. 서론\n주소는 사람이 사는 곳이나 기관, 회사 

In [ ]:
len(data)

889

## Embedding하기

In [8]:
## 메모리 에러나는 코드

ids = []
metadatas = []
embeddings = []

for row in tqdm(data):
    uuid_val = uuid.uuid1()
    index = str(uuid_val)


    tokens = tokenizer.tokenize(row.page_content)
    inputs = tokenizer(row.page_content, return_tensors="pt")
    model_output = model(**inputs)
    
    ids.append(str(index))
    metadatas.append(row.metadata)
    embeddings.append(model_output)

  7%|▋         | 58/889 [01:12<16:03,  1.16s/it]

: 

* torch.no_grad()를 사용하여 gradient 계산을 비활성화하여 메모리 사용량을 줄이기 (학습과정에서는 gradient 계산이 필요하지만, 현재(예측)는 필요 없으므로 생략)
* 모델 출력에서 임베딩을 추출하는 방법을 조정. 모델 출력에 last_hidden_state 속성이 있다고 가정하고 진행(?)

In [35]:
## embedding 하기

ids = []
metadatas = []
embeddings = []

for row in tqdm(data):
    # 고유 식별자 생성
    uuid_val = uuid.uuid1()
    index = str(uuid_val)

    # 입력 텍스트를 토크나이즈 및 인코딩
    tokens = tokenizer.tokenize(row.page_content)
    inputs = tokenizer(row.page_content, return_tensors="pt")

    # 입력 텐서를 모델이 있는 장치로 이동 (필요한 경우)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # 모델 출력 가져오기
    with torch.no_grad():  # 메모리 절약을 위해 gradient 계산 비활성화
        model_output = model(**inputs)
    
    # 임베딩 추출 (모델의 출력 형태에 따라 조정 필요)
    embedding = model_output.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    
    # 결과 리스트에 추가
    ids.append(index)
    metadatas.append(row.metadata)
    embeddings.append(embedding)

100%|██████████| 889/889 [04:02<00:00,  3.67it/s]


In [10]:
embeddings[0]

array([-1.14778489e-01, -3.78033042e-01, -6.53745770e-01, -6.96308613e-01,
        1.22288883e-01, -5.26116155e-02, -8.72128606e-01,  3.93491536e-01,
       -4.28539544e-01,  6.32386088e-01,  3.94339740e-01, -2.15705350e-01,
        1.37642831e-01,  6.12748563e-01,  2.46041685e-01, -5.44937670e-01,
       -2.70692229e-01, -1.39594331e-01, -3.82179230e-01,  1.09031685e-01,
        1.50059074e-01,  2.72650510e-01,  1.62263572e-01,  2.84567297e-01,
        9.43453386e-02,  2.38951951e-01, -5.06899238e-01,  2.13928416e-01,
       -2.79111803e-01,  6.95546985e-01,  1.36778152e+00,  2.50262409e-01,
       -2.59711236e-01,  2.45918751e-01,  2.49131739e-01,  1.03628851e-01,
        4.74568695e-01,  3.74343872e-01,  7.57091865e-02,  2.12693334e-01,
        3.18596303e-01,  7.08310753e-02,  1.18648872e-01,  5.94580895e-04,
       -5.96101582e-01,  8.14902961e-01, -1.33824736e-01,  9.83288884e-01,
       -1.82412446e-01,  4.02918160e-01, -6.30360305e-01,  1.77890480e-01,
       -7.64016733e-02, -

In [36]:
len(embeddings)

889

### Collection에 저장하기 

In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=200)
docs = text_splitter.split_documents(data)

# embeddings = [ e.tolist() for e in tqdm(embeddings)]  

for doc, embed in tqdm(zip(docs, embeddings)):
    uuid_val = uuid.uuid1()
    collection.add(ids=[str(uuid_val)], documents=doc.page_content, metadatas = doc.metadata, embeddings=embed)

889it [00:01, 584.36it/s]


In [41]:
collection.peek()

{'ids': ['e93f04a6-3dbc-11ef-810e-ea9d1e35f997',
  'e940c37c-3dbc-11ef-810e-ea9d1e35f997',
  'e9411660-3dbc-11ef-810e-ea9d1e35f997',
  'e941727c-3dbc-11ef-810e-ea9d1e35f997',
  'e941bcbe-3dbc-11ef-810e-ea9d1e35f997',
  'e9422c26-3dbc-11ef-810e-ea9d1e35f997',
  'e942661e-3dbc-11ef-810e-ea9d1e35f997',
  'e9429d6e-3dbc-11ef-810e-ea9d1e35f997',
  'e942e4fe-3dbc-11ef-810e-ea9d1e35f997',
  'e943227a-3dbc-11ef-810e-ea9d1e35f997'],
 'embeddings': [[-0.11477848887443542,
   -0.3780330419540405,
   -0.6537457704544067,
   -0.6963086128234863,
   0.12228888273239136,
   -0.052611615508794785,
   -0.8721286058425903,
   0.3934915363788605,
   -0.42853954434394836,
   0.6323860883712769,
   0.39433974027633667,
   -0.21570535004138947,
   0.13764283061027527,
   0.6127485632896423,
   0.2460416853427887,
   -0.5449376702308655,
   -0.27069222927093506,
   -0.13959433138370514,
   -0.38217923045158386,
   0.10903168469667435,
   0.1500590741634369,
   0.27265051007270813,
   0.16226357221603394,
   

## Child Docs 생성
* BM25를 위한 Docs
* Collection에 저장했을 때와 똑같은 chunk_size, overlap_size로 설정하기

In [42]:
import uuid
import chromadb
import uuid
from tqdm import tqdm
from langchain.text_splitter import RecursiveCharacterTextSplitter

id_key = "doc_id"
doc_ids = [str(uuid.uuid4()) for _ in data]
child_text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 200)

child_docs = []  # 하위 문서를 저장할 리스트를 초기화합니다.
for i, doc in enumerate(data):
    _id = doc_ids[i]  # 현재 문서의 ID를 가져옵니다.
    # 현재 문서를 하위 문서로 분할합니다.
    child_doc = child_text_splitter.split_documents([doc])
    for _doc in child_doc:  # 분할된 하위 문서에 대해 반복합니다.
        # 하위 문서의 메타데이터에 ID를 저장합니다.
        _doc.metadata[id_key] = _id
    child_docs.extend(child_doc)  # 분할된 하위 문서를 리스트에 추가합니다.

print(len(child_docs))
child_docs

2261


[Document(page_content='Journal of Cadastre & Land InformatiX Vol.52 No.1 (2022)   57\n주소의 국제표준화 동향분석 및 시사점*\n- ISO 19160-2를 중심으로 -\nThe trend of international address standardization and implications\n- with a focus on ISO 19160-2 -\n김성훈** · 김보은*** · 원석환****\nKim, Sung-Hoon · Kim, Bo-Eun · Won, Seok-Hwan\nAbstract\n \nThis study analyzed the status of international address standardization and reviewed responses \nof nations contained in the ISO/CD 19160-2-N5652 standard document. It identified problems in', metadata={'source': '../data/김성훈-주소의 국제표준화 동향분석 및 시사점-2022.pdf', 'file_path': '../data/김성훈-주소의 국제표준화 동향분석 및 시사점-2022.pdf', 'page': 0, 'total_pages': 12, 'format': 'PDF 1.4', 'title': '<342DC1D6BCD2C0C720B1B9C1A6C7A5C1D8C8AD20B5BFC7E2BAD0BCAE20B9D720BDC3BBE7C1A12E687770>', 'author': 'PC', 'subject': '', 'keywords': '', 'creator': 'PScript5.dll Version 5.2.2', 'producer': 'Acrobat Distiller 9.0.0 (Windows)', 'creationDate': 

## Retrieval

### vanila ensemble retrieval
* BM25 + chromadb(similarity)

In [61]:
query="건물번호의 규칙에 대해서 설명해줘"

######################## BM25 retrieval ########################## 

bm25_retriever = BM25Retriever.from_documents(
    child_docs
)
bm25_retriever.k = 1  # BM25Retriever의 검색 결과 개수를 1로 설정합니다.
bm25_result = bm25_retriever.invoke(query)

###################### DB retrieval ###########################

tokens = tokenizer.tokenize(query)
inputs = tokenizer(query, return_tensors="pt")

# 입력 텐서를 모델이 있는 장치로 이동 (필요한 경우)
inputs = {k: v.to(model.device) for k, v in inputs.items()}

# gradient 계산을 비활성화하여 모델 출력 얻기
with torch.no_grad():
    model_output = model(**inputs)

# 모델 출력에서 임베딩 벡터 추출 (여기서는 첫 번째 hidden state의 평균값을 사용)
embedding = model_output.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

# 임베딩 벡터를 리스트 형식으로 변환
embedding_list = embedding.tolist()

result = collection.query(
    query_embeddings=[embedding_list],
    n_results=1
)

In [62]:
print(bm25_result[0].page_content)

러 개 주소부여위치가 있다. 지진옥외대피소의 정문과 후문에 각각 주소
가 부여될 수 있는 것을 예로 들 수 있다. 주소부여위치는 도로연결선과 
반드시 연결되어야 하지만 도로연결선은 주소참조저장소와 반드시 연결
되지는 않도록 하여 현실을 반영하였다. (그림5-1)
이 규칙에 따라 사물DB의 도형 및 속성정보 모델링 결과는 표5-1과 
같다. 단독 시설물과 같은 사물은 포인트로 표현하고 사람들의 활동이 발
10) 사물주소 부여 규칙에 사용된 용어들은 ISO19160-1 주소개념에서 사용하는 개념을 
사물주소에 맞추어 변경하였다.


In [64]:
print(result['documents'][0][0])s

부 연장기한을 정하여 비용 납부를 독촉해야 한다.
⑥ 시장등은 제5항에 따른 납부 연장기한까지 정비비용을 납부하지 않은 경우에는 법 제26조제6항에 따라 이를 징
수할 수 있다.
 
제50조(각종 개발사업에 따른 주소정보시설의 설치ㆍ교체 등) ① 법 제26조제4항에 따른 도시개발사업 및 주택재개발
사업 등 각종 개발사업의 시행자(이하 이 조에서 “개발사업시행자”라 한다)는 그 개발사업으로 주소정보시설의 설치
ㆍ교체 또는 철거가 필요한 경우에는 그 개발사업을 수행하기 위한 인허가 또는 승인을 신청할 때 다음 각 호의 사


In [63]:
result

{'ids': [['e9d50da2-3dbc-11ef-810e-ea9d1e35f997']],
 'distances': [[162.96597290039062]],
 'metadatas': [[{'author': '',
    'creationDate': "D:20240608104108+09'00'",
    'creator': '',
    'file_path': '../data/도로명주소법 시행령(대통령령)(제34533호)(20240528).pdf',
    'format': 'PDF 1.4',
    'keywords': '',
    'modDate': "D:20240608104108+09'00'",
    'page': 18,
    'producer': 'iText 2.1.7 by 1T3XT',
    'source': '../data/도로명주소법 시행령(대통령령)(제34533호)(20240528).pdf',
    'subject': '',
    'title': '',
    'total_pages': 25,
    'trapped': ''}]],
 'embeddings': None,
 'documents': [['부 연장기한을 정하여 비용 납부를 독촉해야 한다.\n⑥ 시장등은 제5항에 따른 납부 연장기한까지 정비비용을 납부하지 않은 경우에는 법 제26조제6항에 따라 이를 징\n수할 수 있다.\n \n제50조(각종 개발사업에 따른 주소정보시설의 설치ㆍ교체 등) ① 법 제26조제4항에 따른 도시개발사업 및 주택재개발\n사업 등 각종 개발사업의 시행자(이하 이 조에서 “개발사업시행자”라 한다)는 그 개발사업으로 주소정보시설의 설치\nㆍ교체 또는 철거가 필요한 경우에는 그 개발사업을 수행하기 위한 인허가 또는 승인을 신청할 때 다음 각 호의 사']],
 'uris': None,
 'data': None}